In [ ]:
import pandas as pd
import numpy as np
from common_variables import *

In [ ]:
df_csv = pd.read_csv(full_time_series_path, usecols=['Date','High','Low'], 
    index_col=['Date'], parse_dates=['Date'])
df = df_csv[df_csv.index >= pd.to_datetime(start_date)]
df

In [ ]:
df["HLAvg"] = df['High'].add(df['Low']).div(2)
del df['High']
del df['Low']
# Simple Moving Average
df['MA'] = df['HLAvg'].rolling(window=ma_periods).mean()
# Log Returns
df['Returns'] = np.log(df['MA']/df['MA'].shift(1))
df.head(15)

In [ ]:
df.dropna(how='any', inplace=True)
df = df[df.shape[0] % batch_size:]
df

In [ ]:
df_train = df[:- validation_size - test_size]
df_validation = df[- validation_size - test_size - window_size:- test_size]
df_test = df[- test_size - window_size:]
print(f'df_train.shape {df_train.shape}, df_validation.shape {df_validation.shape}, df_test.shape {df_test.shape}')

In [ ]:
df_train.to_csv(train_time_series_path)
df_validation.to_csv(validate_time_series_path)
df_test.to_csv(test_time_series_path)